In [1]:
import pandas as pd
import os 
from os import listdir
from os.path import isfile, join 
import glob
import re
import matplotlib 
from matplotlib import pyplot as plt
import seaborn as sns
import io
from scipy.stats import ttest_ind
from scipy import stats
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import f_oneway




In [2]:
path=os.getcwd()
print(path)
from google.colab import drive
drive.mount('/content/drive')

/content
Mounted at /content/drive


In [3]:
df = pd.ExcelFile('/content/drive/My Drive/Arrests.xlsx')

In [4]:
table1 = pd.read_excel(df,'Arrests_and_Strip_Searches_(RBD')
table1.head()

,Arrest_Year,Arrest_Month,EventID,ArrestID,PersonID,Perceived_Race,Sex,Age_group__at_arrest_,Youth_at_arrest__under_18_years,ArrestLocDiv,...,Actions_at_arrest___Resisted__d,Actions_at_arrest___Mental_inst,Actions_at_arrest___Assaulted_o,Actions_at_arrest___Cooperative,SearchReason_CauseInjury,SearchReason_AssistEscape,SearchReason_PossessWeapons,SearchReason_PossessEvidence,ItemsFound,ObjectId
0,2020,July-Sept,1005907,6017884.0,326622,White,M,Aged 35 to 44 years,Not a youth,54,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,1
1,2020,July-Sept,1014562,6056669.0,326622,White,M,Aged 35 to 44 years,Not a youth,54,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2
2,2020,Oct-Dec,1029922,6057065.0,326622,Unknown or Legacy,M,Aged 35 to 44 years,Not a youth,54,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,3
3,2021,Jan-Mar,1052190,6029059.0,327535,Black,M,Aged 25 to 34 years,Not a youth,XX,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4
4,2021,Jan-Mar,1015512,6040372.0,327535,South Asian,M,Aged 25 to 34 years,Not a youth,XX,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,5


In [5]:
# Convert male to 1 and female to zero
table1['Sex'] = table1['Sex'].apply(lambda x: 1 if x == 'M' else 0)

In [6]:
# convert black to 1 and everything else to zero 
table1['Perceived_Race'] = table1['Perceived_Race'].apply(lambda x: 1 if x == 'Black' else 0)

In [7]:
table1.head()

,Arrest_Year,Arrest_Month,EventID,ArrestID,PersonID,Perceived_Race,Sex,Age_group__at_arrest_,Youth_at_arrest__under_18_years,ArrestLocDiv,...,Actions_at_arrest___Resisted__d,Actions_at_arrest___Mental_inst,Actions_at_arrest___Assaulted_o,Actions_at_arrest___Cooperative,SearchReason_CauseInjury,SearchReason_AssistEscape,SearchReason_PossessWeapons,SearchReason_PossessEvidence,ItemsFound,ObjectId
0,2020,July-Sept,1005907,6017884.0,326622,0,1,Aged 35 to 44 years,Not a youth,54,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,1
1,2020,July-Sept,1014562,6056669.0,326622,0,1,Aged 35 to 44 years,Not a youth,54,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2
2,2020,Oct-Dec,1029922,6057065.0,326622,0,1,Aged 35 to 44 years,Not a youth,54,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,3
3,2021,Jan-Mar,1052190,6029059.0,327535,1,1,Aged 25 to 34 years,Not a youth,XX,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4
4,2021,Jan-Mar,1015512,6040372.0,327535,0,1,Aged 25 to 34 years,Not a youth,XX,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,5


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# logistic regression
X = table1[['Perceived_Race', 'Sex']]
y = table1['Booked']

# Slit data into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the logistic regression model and fit it to the training data
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# get class labels 
y_pred = logreg.predict(X_test)

# get accuracy of model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5502450980392157


In [9]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols



# Perform ANCOVA with 'Race' and 'Sex' as independent variables and 'booked' as the dependent variable
model = ols('Booked ~ Perceived_Race + Sex', table1).fit()

# Print  table
print(sm.stats.anova_lm(model, typ=2))


                      sum_sq       df           F        PR(>F)
Perceived_Race     21.742418      1.0   87.815860  7.404166e-21
Sex               108.070720      1.0  436.488856  1.304860e-96
Residual        16161.008483  65273.0         NaN           NaN


In [10]:
# Perform ANCOVA with  'StripSearch' as the continuous covariate
model = ols('Booked ~ Perceived_Race + Sex + StripSearch ', table1).fit()

# Print  table
print(sm.stats.anova_lm(model, typ=2))

                      sum_sq       df            F        PR(>F)
Perceived_Race     11.028680      1.0    48.891973  2.730430e-12
Sex                88.592377      1.0   392.744743  3.777482e-87
StripSearch      1437.446372      1.0  6372.438877  0.000000e+00
Residual        14723.562111  65272.0          NaN           NaN


In [11]:
import statsmodels.stats.power as smp

alpha = 0.05   
power = 0.8    
effect_size = 0.5  

# power analysis
power_analysis = smp.TTestIndPower()
sample_size = power_analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power)

print(f"Sample size: {sample_size}")


Sample size: 63.765610587854056


/usr/local/lib/python3.9/dist-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: invalid value encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/usr/local/lib/python3.9/dist-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: invalid value encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)
